In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
PLR_gdf = gpd.read_file('../data/input/berlin/lor_planungsraeume_2021.geojson') # import Planungsräume (PLR) geo data
BZR_gdf = gpd.read_file('../data/input/berlin/lor_bezirksregionen_2021.geojson') # import Bezirksregionen (BZR) geo data
PGR_gdf = gpd.read_file('../data/input/berlin/lor_prognoseraeume_2021.geojson') # import Prognoseräume (PGR) geo data
STREETS_gdf = gpd.read_file('../data/input/berlin/Detailnetz-Strassenabschnitte.geojson') # import Straßen geo data

In [3]:
PLR_gdf.head(5)

,PLR_ID,PLR_NAME,BEZ,STAND,GROESSE_M2,geometry
0,11501341,Karlshorst Süd,11,01.01.2021,2294021.723,"MULTIPOLYGON (((400826.929 5814871.126, 400825..."
1,07601340,Tirschenreuther Ring Ost,07,01.01.2021,413969.326,"MULTIPOLYGON (((387795.701 5807587.153, 387789..."
2,02500831,Wismarplatz,02,01.01.2021,248988.291,"MULTIPOLYGON (((396010.552 5819339.570, 396021..."
3,12601134,Märkisches Zentrum,12,01.01.2021,1127855.841,"MULTIPOLYGON (((387367.428 5828619.763, 387315..."
4,07601547,Horstwalder Straße,07,01.01.2021,941433.954,"MULTIPOLYGON (((391807.174 5805331.900, 391808..."


In [4]:
accident_df = pd.read_csv('../data/temp/raw_accident_dataset_2018-2021.csv')

C:\Users\Domen\AppData\Local\Temp\ipykernel_43288\767954756.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  accident_df = pd.read_csv('../data/temp/raw_accident_dataset_2018-2021.csv')


In [5]:
# accident_df.head(5).T

In [6]:
accident_df = accident_df.drop(columns=["lor", "linrefx", "linrefy"])

In [7]:
accident_gdf = gpd.GeoDataFrame(accident_df, geometry=gpd.points_from_xy(accident_df.longitude, accident_df.latitude),
    crs="EPSG:4326") # transform to GeoDataFrame

### Add PLR Area

In [8]:
accident_gdf = accident_gdf.to_crs(PLR_gdf.crs) # same crs
joined_gdf = gpd.sjoin(accident_gdf, PLR_gdf, op='within', how='left') # join

C:\Users\Domen\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [9]:
accident_df['PLR_ID'] = joined_gdf['PLR_ID'] # add to main df
accident_df['PLR_NAME'] = joined_gdf['PLR_NAME'] # add to main df


### Add BZR area

In [10]:
accident_gdf = accident_gdf.to_crs(BZR_gdf.crs) # same crs
joined_gdf = gpd.sjoin(accident_gdf, BZR_gdf, predicate='within', how='left') # join

In [11]:
accident_df['BZR_ID'] = joined_gdf['BZR_ID'] # add to main df
accident_df['BZR_NAME'] = joined_gdf['BZR_NAME'] # add to main df


#### ADD PGR area

In [12]:
accident_gdf = accident_gdf.to_crs(PGR_gdf.crs) # same crs
joined_gdf = gpd.sjoin(accident_gdf, PGR_gdf, predicate='within', how='left') # join

In [13]:
accident_df['PGR_ID'] = joined_gdf['PGR_ID'] # add to main df
accident_df['PGR_NAME'] = joined_gdf['PGR_NAME'] # add to main df


In [14]:
joined_gdf

,key,objectid,bez,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,...,ustrzustand,longitude,latitude,geometry,index_right,PGR_ID,PGR_NAME,BEZ,STAND,GROESSE_M2
0,112695-2018,112695,2,Samariterviertel,2500729,2018,1,15,4,3,...,1,13.475018,52.513597,POINT (396514.624 5819257.316),16.0,0250,Friedrichshain Ost,02,01.01.2021,5.371447e+06
1,112705-2018,112705,12,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,...,0,13.291022,52.587259,POINT (384223.627 5827729.725),53.0,1250,Wittenau/Borsigwalde,12,01.01.2021,1.587637e+07
2,112726-2018,112726,2,Barnimkiez,2400520,2018,1,9,3,3,...,0,13.420578,52.526019,POINT (392850.817 5820718.312),26.0,0240,Friedrichshain West,02,01.01.2021,4.578339e+06
3,112737-2018,112737,7,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,...,0,13.348288,52.481844,POINT (387834.522 5815915.019),32.0,0720,Schöneberg Süd,07,01.01.2021,3.358749e+06
4,112747-2018,112747,3,Niederschönhausen,3200206,2018,1,15,4,3,...,1,13.403228,52.583472,POINT (391815.267 5827134.254),52.0,0320,Nördliches Pankow,03,01.01.2021,2.613445e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50114,112174-2021,112174,4,0,4300414,2021,3,16,4,3,...,1,13.296345,52.511008,POINT (384383.944 5819240.530),17.0,0430,Charlottenburg Zentrum,04,01.01.2021,9.292147e+06
50115,111541-2021,111541,4,0,4400726,2021,1,1,6,3,...,0,13.289496,52.493813,POINT (383873.882 5817339.107),6.0,0440,Wilmersdorf Süd,04,01.01.2021,1.166820e+07
50116,110370-2021,110370,3,0,2400520,2021,1,20,1,3,...,1,13.423095,52.527534,POINT (393025.206 5820883.134),26.0,0240,Friedrichshain West,02,01.01.2021,4.578339e+06
50117,109998-2021,109998,7,0,7300619,2021,1,14,7,3,...,1,13.359259,52.474367,POINT (388560.601 5815066.337),5.0,0730,Friedenau,07,01.01.2021,4.603888e+06


In [15]:
accident_df

,key,objectid,bez,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,...,istsonstige,ustrzustand,longitude,latitude,PLR_ID,PLR_NAME,BZR_ID,BZR_NAME,PGR_ID,PGR_NAME
0,112695-2018,112695,2,Samariterviertel,2500729,2018,1,15,4,3,...,0,1,13.475018,52.513597,02500729,Pettenkofer Straße,025007,Frankfurter Allee Nord,0250,Friedrichshain Ost
1,112705-2018,112705,12,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,...,0,0,13.291022,52.587259,12500824,Ziekowstraße/Freie Scholle,125008,West 3 - Borsigwalde/Freie Scholle,1250,Wittenau/Borsigwalde
2,112726-2018,112726,2,Barnimkiez,2400520,2018,1,9,3,3,...,0,0,13.420578,52.526019,02400520,Barnimkiez,024005,Karl-Marx-Allee Nord,0240,Friedrichshain West
3,112737-2018,112737,7,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,...,0,0,13.348288,52.481844,07200308,Volkspark (Rudolph-Wilde-Park),072003,Schöneberg Südwest,0720,Schöneberg Süd
4,112747-2018,112747,3,Niederschönhausen,3200206,2018,1,15,4,3,...,0,1,13.403228,52.583472,03200206,Pastor-Niemöller-Platz,032002,Blankenfelde/Niederschönhausen,0320,Nördliches Pankow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50114,112174-2021,112174,4,0,4300414,2021,3,16,4,3,...,0,1,13.296345,52.511008,04300414,Schloßstraße,043004,Schloss Charlottenburg,0430,Charlottenburg Zentrum
50115,111541-2021,111541,4,0,4400726,2021,1,1,6,3,...,0,0,13.289496,52.493813,04400726,Bismarckallee,044007,Grunewald,0440,Wilmersdorf Süd
50116,110370-2021,110370,3,0,2400520,2021,1,20,1,3,...,0,1,13.423095,52.527534,02400520,Barnimkiez,024005,Karl-Marx-Allee Nord,0240,Friedrichshain West
50117,109998-2021,109998,7,0,7300619,2021,1,14,7,3,...,0,1,13.359259,52.474367,07300619,Grazer Platz,073006,Friedenau Ost,0730,Friedenau


#### OSMNX Method

In [21]:
accident_df.head(5).T

,0,1,2,3,4
key,112695-2018,112705-2018,112726-2018,112737-2018,112747-2018
objectid,112695,112705,112726,112737,112747
bez,2,12,2,7,3
strasse,Samariterviertel,Ziekowstraße/Freie Scholle,Barnimkiez,Volkspark (Rudolf-Wilde-Park),Niederschönhausen
lor_ab_2021,2500729,12500824,2400520,7200308,3200206
ujahr,2018,2018,2018,2018,2018
umonat,1,1,1,1,1
ustunde,15,11,9,17,15
uwochentag,4,2,3,2,4
ukategorie,3,3,3,3,3


#### nearest distance

In [28]:
STREETS_gdf

,element_nr,strassensc,strassenna,str_bez,strassenkl,strassen_1,strassen_2,verkehrsri,bezirk,stadtteil,verkehrseb,beginnt_be,endet_bei_,laenge,gueltig_vo,okstra_id,geometry
0,34610003_34610004.01,00002,Aalemannufer,NaN,IV,G,STRA,B,Spandau,Hakenfelde,0,34610003,34610004,262.5000,2010-01-01,D62521E5E27544729878420C54E6C59C,"MULTILINESTRING ((13.21996 52.57307, 13.22225 ..."
1,40540001_41540003.01,00005,Abbestraße,NaN,V,G,STRA,B,Charlottenburg-Wilmersdorf,Charlottenburg,0,40540001,41540003,182.4500,2010-01-01,275EE05309AF45DCA49E046BBA0CBBCC,"MULTILINESTRING ((13.31987 52.51527, 13.32117 ..."
2,42590002_42590001.01,00022,Afrikanische Straße,NaN,II,G,STRA,B,Mitte,Wedding,0,42590002,42590001,65.3600,2010-01-01,18F55F73EB5346F6A7A719E970B9D4EC,"MULTILINESTRING ((13.33344 52.56096, 13.33332 ..."
3,42590003_42590004.01,00022,Afrikanische Straße,NaN,II,G,STRA,B,Mitte,Wedding,0,42590003,42590004,61.4100,2010-01-01,AE661AB3DB344DD183550F05E15A4BD7,"MULTILINESTRING ((13.33431 52.55736, 13.33458 ..."
4,42590005_42590006.01,00022,Afrikanische Straße,NaN,II,G,STRA,B,Mitte,Wedding,0,42590005,42590006,45.3600,2010-01-01,D47D9FE02F13481FA79966646BA478FA,"MULTILINESTRING ((13.33481 52.55642, 13.33483 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43105,53530049_53530051.01,41423,Irenenstraße,NaN,V,G,STRA,B,Lichtenberg,Rummelsburg,0,53530049,53530051,192.8000,2021-03-04,BBA9B5EF7DB242898DDE1E53868D9227,"MULTILINESTRING ((13.49975 52.50937, 13.50007 ..."
43106,53530051_53530040.01,42456,Rosenfelder Straße,NaN,V,G,STRA,B,Lichtenberg,Rummelsburg,0,53530051,53530040,123.1700,2021-03-04,92AD8579703C4E12B378108ED328B192,"MULTILINESTRING ((13.50249 52.50890, 13.50258 ..."
43107,53530061_53530040.01,40696,Einbecker Straße,NaN,III,G,STRA,B,Lichtenberg,Rummelsburg,0,53530061,53530040,159.5962,2021-03-04,D8CB50FB08E34C0A8AA1A0095A40B0EF,"MULTILINESTRING ((13.50141 52.51033, 13.50335 ..."
43108,53530057_53530061.01,40696,Einbecker Straße,NaN,III,G,STRA,B,Lichtenberg,Rummelsburg,0,53530057,53530061,52.6011,2021-03-04,2EB39A91A5AB43B3B612EC86DE4A3DD6,"MULTILINESTRING ((13.50073 52.51056, 13.50127 ..."


In [29]:
# Create a Point geometry from latitude and longitude for each accident
accident_df['geometry'] = accident_df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

# Convert the DataFrame to a GeoDataFrame
accident_gdf = gpd.GeoDataFrame(accident_df, geometry='geometry')
accident_gdf

,key,objectid,bez,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,...,ustrzustand,longitude,latitude,PLR_ID,PLR_NAME,BZR_ID,BZR_NAME,PGR_ID,PGR_NAME,geometry
0,112695-2018,112695,2,Samariterviertel,2500729,2018,1,15,4,3,...,1,13.475018,52.513597,02500729,Pettenkofer Straße,025007,Frankfurter Allee Nord,0250,Friedrichshain Ost,POINT (13.47502 52.51360)
1,112705-2018,112705,12,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,...,0,13.291022,52.587259,12500824,Ziekowstraße/Freie Scholle,125008,West 3 - Borsigwalde/Freie Scholle,1250,Wittenau/Borsigwalde,POINT (13.29102 52.58726)
2,112726-2018,112726,2,Barnimkiez,2400520,2018,1,9,3,3,...,0,13.420578,52.526019,02400520,Barnimkiez,024005,Karl-Marx-Allee Nord,0240,Friedrichshain West,POINT (13.42058 52.52602)
3,112737-2018,112737,7,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,...,0,13.348288,52.481844,07200308,Volkspark (Rudolph-Wilde-Park),072003,Schöneberg Südwest,0720,Schöneberg Süd,POINT (13.34829 52.48184)
4,112747-2018,112747,3,Niederschönhausen,3200206,2018,1,15,4,3,...,1,13.403228,52.583472,03200206,Pastor-Niemöller-Platz,032002,Blankenfelde/Niederschönhausen,0320,Nördliches Pankow,POINT (13.40323 52.58347)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50114,112174-2021,112174,4,0,4300414,2021,3,16,4,3,...,1,13.296345,52.511008,04300414,Schloßstraße,043004,Schloss Charlottenburg,0430,Charlottenburg Zentrum,POINT (13.29634 52.51101)
50115,111541-2021,111541,4,0,4400726,2021,1,1,6,3,...,0,13.289496,52.493813,04400726,Bismarckallee,044007,Grunewald,0440,Wilmersdorf Süd,POINT (13.28950 52.49381)
50116,110370-2021,110370,3,0,2400520,2021,1,20,1,3,...,1,13.423095,52.527534,02400520,Barnimkiez,024005,Karl-Marx-Allee Nord,0240,Friedrichshain West,POINT (13.42309 52.52753)
50117,109998-2021,109998,7,0,7300619,2021,1,14,7,3,...,1,13.359259,52.474367,07300619,Grazer Platz,073006,Friedenau Ost,0730,Friedenau,POINT (13.35926 52.47437)


In [30]:
# Set CRS for accident data to WGS 84
accident_gdf = accident_gdf.set_crs('EPSG:4326')

# Convert accident data to the same CRS as the street data
accident_gdf = accident_gdf.to_crs(STREETS_gdf.crs)


In [31]:
# Check for invalid geometries in accidents
invalid_accidents = accident_gdf[~accident_gdf.is_valid]
print("Invalid accident geometries:", invalid_accidents)

# Check for invalid geometries in streets
invalid_streets = STREETS_gdf[~STREETS_gdf.is_valid]
print("Invalid street geometries:", invalid_streets)

Invalid accident geometries: Empty GeoDataFrame
Columns: [key, objectid, bez, strasse, lor_ab_2021, ujahr, umonat, ustunde, uwochentag, ukategorie, uart, utyp1, ulichtverh, istrad, istpkw, istfuss, istkrad, istgkfz, istsonstige, ustrzustand, longitude, latitude, PLR_ID, PLR_NAME, BZR_ID, BZR_NAME, PGR_ID, PGR_NAME, geometry]
Index: []

[0 rows x 29 columns]
Invalid street geometries: Empty GeoDataFrame
Columns: [element_nr, strassensc, strassenna, str_bez, strassenkl, strassen_1, strassen_2, verkehrsri, bezirk, stadtteil, verkehrseb, beginnt_be, endet_bei_, laenge, gueltig_vo, okstra_id, geometry]
Index: []


#### Using Street data from berlin website

In [ ]:
# Select the first 10 rows (or any other number you prefer)
accident_subset = accident_gdf.head(10)

In [33]:
from shapely.ops import nearest_points
from tqdm import tqdm

# Initialize columns to store the nearest street information
accident_subset['nearest_street_idx'] = None
accident_subset['street_attribute'] = None

# For each accident point, find the nearest street
for index, row in tqdm(accident_subset.iterrows()):
    point = row.geometry
    
    # Find the nearest street geometry and its index
    nearest_geom, nearest_idx = None, None
    min_distance = float('inf')
    for idx, street_row in STREETS_gdf.iterrows():
        street_geom = street_row.geometry
        distance = point.distance(street_geom)
        if distance < min_distance:
            min_distance = distance
            nearest_geom = street_geom
            nearest_idx = idx
    
    # Update the GeoDataFrame with the nearest street information
    accident_subset.at[index, 'nearest_street_idx'] = nearest_idx
    accident_subset.at[index, 'street_attribute'] = STREETS_gdf.at[nearest_idx, 'strassenna']


C:\Users\Domen\anaconda3\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Domen\anaconda3\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
0it [00:00, ?it/s]C:\Users\Domen\anaconda3\lib\site-packages\shapely\measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
1it [00:03,  3.23s/it]C:\U

In [34]:
accident_subset

,key,objectid,bez,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,...,latitude,PLR_ID,PLR_NAME,BZR_ID,BZR_NAME,PGR_ID,PGR_NAME,geometry,nearest_street_idx,street_attribute
0,112695-2018,112695,2,Samariterviertel,2500729,2018,1,15,4,3,...,52.513597,02500729,Pettenkofer Straße,025007,Frankfurter Allee Nord,0250,Friedrichshain Ost,POINT (13.47502 52.51360),6209,Frankfurter Allee
1,112705-2018,112705,12,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,...,52.587259,12500824,Ziekowstraße/Freie Scholle,125008,West 3 - Borsigwalde/Freie Scholle,1250,Wittenau/Borsigwalde,POINT (13.29102 52.58726),41373,A 111 BAB Autobahnzubringer Hamburg
2,112726-2018,112726,2,Barnimkiez,2400520,2018,1,9,3,3,...,52.526019,02400520,Barnimkiez,024005,Karl-Marx-Allee Nord,0240,Friedrichshain West,POINT (13.42058 52.52602),4410,Otto-Braun-Straße
3,112737-2018,112737,7,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,...,52.481844,07200308,Volkspark (Rudolph-Wilde-Park),072003,Schöneberg Südwest,0720,Schöneberg Süd,POINT (13.34829 52.48184),828,Dominicusstraße
4,112747-2018,112747,3,Niederschönhausen,3200206,2018,1,15,4,3,...,52.583472,03200206,Pastor-Niemöller-Platz,032002,Blankenfelde/Niederschönhausen,0320,Nördliches Pankow,POINT (13.40323 52.58347),21397,Dietzgenstraße
5,112770-2018,112770,10,Ringkolonnaden,10100208,2018,1,5,2,2,...,52.551095,10100208,Bürgerpark,101002,Marzahn Mitte,1010,Marzahn,POINT (13.54845 52.55109),7709,Märkische Allee
6,112792-2018,112792,4,Forst Grunewald,4200207,2018,1,17,2,3,...,52.485715,04200207,Eichkamp,042002,Heerstraße,0420,Charlottenburg West,POINT (13.25639 52.48572),41696,A 115 BAB Autobahnzubringer Magdebg/Leipz (Avus)
7,112805-2018,112805,3,Lietzengraben,3100101,2018,1,16,5,3,...,52.633823,03100101,Bucher Forst,031001,Buch,0310,Buch,POINT (13.42725 52.63382),42255,A 114 BAB Autobahnzubringer Prenzlau
8,112820-2018,112820,1,Alexanderplatzviertel,1100310,2018,1,14,2,3,...,52.518933,01100310,Alexanderplatzviertel,011003,Alexanderplatz,0110,Zentrum,POINT (13.41143 52.51893),8333,Grunerstraße
9,112837-2018,112837,12,Teichstraße,12100205,2018,1,21,7,3,...,52.573889,12100205,Teichstraße,121002,Ost 2 - Alt-Reinickendorf,1210,Reinickendorf Ost,POINT (13.35020 52.57389),2201,Lindauer Allee


In [35]:
street_sindex = STREETS_gdf.sindex

#### OSMNX geo data 

In [37]:
# # Import necessary libraries
# from shapely.geometry import Point
# from geopandas.tools import sjoin
# import osmnx as ox
# import geopandas as gpd

# # Fetch the road network of Berlin
# G = ox.graph_from_place('Berlin, Germany', network_type='drive')

# # Split the road network into nodes and edges
# nodes, edges = ox.graph_to_gdfs(G)

In [38]:
# ox.plot_graph(G)

In [39]:
# from tqdm import tqdm
# import numpy as np
# import pandas as pd
# from shapely.geometry import Point


# tqdm.pandas()

# ## funktioniert

# def find_nearest_street(row):
#     point = Point(row['longitude'], row['latitude'])
    
#     # Use the spatial index to find the nearest edges
#     possible_matches_index = np.array(list(edges.sindex.nearest(point, return_all=True))).flatten()
    
#     # Use .iloc to select the candidates
#     possible_matches = edges.iloc[possible_matches_index]
    
#     # Find the nearest edge (more expensive geometric operation)
#     nearest_edge = possible_matches.geometry.distance(point).idxmin()
    
#     # Get the street name
#     street_name = edges.loc[nearest_edge]['name']
#     return street_name


# # Function to split a dataframe into chunks
# def split_dataframe(df, chunk_size):
#     chunks = [df[i:i + chunk_size] for i in range(0, df.shape[0], chunk_size)]
#     return chunks

# # Split the dataframe
# chunk_size = len(accident_df) // 10
# data_chunks = split_dataframe(accident_df, chunk_size)

# # Process each chunk and save the results to a file
# for i, chunk in enumerate(data_chunks, start=1):
#     print(f"Processing chunk {i}/{len(data_chunks)}")
#     chunk['nearest_street'] = chunk.progress_apply(find_nearest_street, axis=1)
#     chunk.to_csv(f'../data/temp/accident_data_chunk_{i}.csv', index=False)
#     print(f"Chunk {i} saved to accident_data_chunk_{i}.csv")

# # Assign the results to the original DataFrame
# all_chunks = [pd.read_csv(f'accident_data_chunk_{i}.csv') for i in range(1, len(data_chunks) + 1)]
# accident_df = pd.concat(all_chunks, ignore_index=True)

In [48]:
# all_chunks = [pd.read_csv(f'accident_data_chunk_{i}.csv') for i in range(1, 11 + 1)]
# accident_df = pd.concat(all_chunks, ignore_index=True)

In [69]:
accident_df

,key,objectid,bez,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,...,longitude,latitude,PLR_ID,PLR_NAME,BZR_ID,BZR_NAME,PGR_ID,PGR_NAME,geometry,nearest_street
0,112695-2018,112695,2,Samariterviertel,2500729,2018,1,15,4,3,...,13.475018,52.513597,2500729.0,Pettenkofer Straße,25007.0,Frankfurter Allee Nord,250.0,Friedrichshain Ost,POINT (13.4750178 52.51359681),Frankfurter Allee
1,112705-2018,112705,12,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,...,13.291022,52.587259,12500824.0,Ziekowstraße/Freie Scholle,125008.0,West 3 - Borsigwalde/Freie Scholle,1250.0,Wittenau/Borsigwalde,POINT (13.29102205 52.58725906),NaN
2,112726-2018,112726,2,Barnimkiez,2400520,2018,1,9,3,3,...,13.420578,52.526019,2400520.0,Barnimkiez,24005.0,Karl-Marx-Allee Nord,240.0,Friedrichshain West,POINT (13.42057818 52.52601854),Otto-Braun-Straße
3,112737-2018,112737,7,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,...,13.348288,52.481844,7200308.0,Volkspark (Rudolph-Wilde-Park),72003.0,Schöneberg Südwest,720.0,Schöneberg Süd,POINT (13.34828776 52.48184447),Hauptstraße
4,112747-2018,112747,3,Niederschönhausen,3200206,2018,1,15,4,3,...,13.403228,52.583472,3200206.0,Pastor-Niemöller-Platz,32002.0,Blankenfelde/Niederschönhausen,320.0,Nördliches Pankow,POINT (13.40322797 52.58347154),Dietzgenstraße
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50114,112174-2021,112174,4,0,4300414,2021,3,16,4,3,...,13.296345,52.511008,4300414.0,Schloßstraße,43004.0,Schloss Charlottenburg,430.0,Charlottenburg Zentrum,POINT (13.29634478 52.51100814),"['Sophie-Charlotte-Platz', 'Kaiserdamm']"
50115,111541-2021,111541,4,0,4400726,2021,1,1,6,3,...,13.289496,52.493813,4400726.0,Bismarckallee,44007.0,Grunewald,440.0,Wilmersdorf Süd,POINT (13.28949592 52.49381321),NaN
50116,110370-2021,110370,3,0,2400520,2021,1,20,1,3,...,13.423095,52.527534,2400520.0,Barnimkiez,24005.0,Karl-Marx-Allee Nord,240.0,Friedrichshain West,POINT (13.42309463 52.52753402),Otto-Braun-Straße
50117,109998-2021,109998,7,0,7300619,2021,1,14,7,3,...,13.359259,52.474367,7300619.0,Grazer Platz,73006.0,Friedenau Ost,730.0,Friedenau,POINT (13.35925879 52.47436651),NaN


In [54]:
accident_df.to_pickle("../data/temp/temp_adress_data.pkl")


In [63]:
accident_df["nearest_street"].value_counts(dropna=False)


nearest_street
NaN                                      1573
Landsberger Allee                         462
Frankfurter Allee                         360
Heerstraße                                319
Kurfürstendamm                            311
                                         ... 
Helene-Weigel-Platz                         1
Ludwig-Richter-Straße                       1
Straße 42                                   1
Saarburger Straße                           1
['Wuhletalstraße', 'Märkische Allee']       1
Name: count, Length: 4582, dtype: int64

In [61]:
nan_entries = accident_df[pd.isna(accident_df["nearest_street"])]

In [62]:
nan_entries

,key,objectid,bez,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,...,longitude,latitude,PLR_ID,PLR_NAME,BZR_ID,BZR_NAME,PGR_ID,PGR_NAME,geometry,nearest_street
1,112705-2018,112705,12,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,...,13.291022,52.587259,12500824.0,Ziekowstraße/Freie Scholle,125008.0,West 3 - Borsigwalde/Freie Scholle,1250.0,Wittenau/Borsigwalde,POINT (13.29102205 52.58725906),NaN
7,112805-2018,112805,3,Lietzengraben,3100101,2018,1,16,5,3,...,13.427252,52.633823,3100101.0,Bucher Forst,31001.0,Buch,310.0,Buch,POINT (13.42725182 52.63382261),NaN
11,112862-2018,112862,7,Manteuffelstraße ...,7400823,2018,1,16,4,3,...,13.377730,52.470103,7400823.0,Wittekindstraße,74008.0,Tempelhof Südwest,740.0,Tempelhof,POINT (13.37773011 52.47010259),NaN
18,112980-2018,112980,7,Manteuffelstraße ...,7400823,2018,1,13,3,3,...,13.377436,52.470119,7400823.0,Wittekindstraße,74008.0,Tempelhof Südwest,740.0,Tempelhof,POINT (13.37743615 52.47011861),NaN
21,113020-2018,113020,12,Ziekowstraße/Freie Scholle,12500824,2018,1,7,6,3,...,13.284104,52.597047,12500824.0,Ziekowstraße/Freie Scholle,125008.0,West 3 - Borsigwalde/Freie Scholle,1250.0,Wittenau/Borsigwalde,POINT (13.28410402 52.5970472),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50094,112216-2021,112216,4,0,4501153,2021,6,8,5,3,...,13.336492,52.478394,4501153.0,Babelsberger Straße,45011.0,Volkspark Wilmersdorf,450.0,Wilmersdorf Zentrum,POINT (13.33649249 52.47839438),NaN
50107,112189-2021,112189,4,0,4400830,2021,9,14,3,3,...,13.306530,52.480177,4400830.0,Forckenbeckstraße,44008.0,Schmargendorf,440.0,Wilmersdorf Süd,POINT (13.30653019 52.48017738),NaN
50108,112187-2021,112187,7,0,7400928,2021,3,15,1,3,...,13.419587,52.461058,7400928.0,Germaniagarten,74009.0,Tempelhof Südost,740.0,Tempelhof,POINT (13.41958744 52.461058),NaN
50115,111541-2021,111541,4,0,4400726,2021,1,1,6,3,...,13.289496,52.493813,4400726.0,Bismarckallee,44007.0,Grunewald,440.0,Wilmersdorf Süd,POINT (13.28949592 52.49381321),NaN


#### TEST

In [71]:
# Create a Point geometry from latitude and longitude for each accident
nan_entries['geometry'] = nan_entries.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

# Convert the DataFrame to a GeoDataFrame
nan_entries_gdf = gpd.GeoDataFrame(nan_entries, geometry='geometry')
nan_entries_gdf

C:\Users\Domen\AppData\Local\Temp\ipykernel_43288\4256007842.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_entries['geometry'] = nan_entries.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)


,key,objectid,bez,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,...,PLR_ID,PLR_NAME,BZR_ID,BZR_NAME,PGR_ID,PGR_NAME,geometry,nearest_street,nearest_street_idx,second_street_search
1,112705-2018,112705,12,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,...,12500824.0,Ziekowstraße/Freie Scholle,125008.0,West 3 - Borsigwalde/Freie Scholle,1250.0,Wittenau/Borsigwalde,POINT (13.29102 52.58726),NaN,NaN,NaN
7,112805-2018,112805,3,Lietzengraben,3100101,2018,1,16,5,3,...,3100101.0,Bucher Forst,31001.0,Buch,310.0,Buch,POINT (13.42725 52.63382),NaN,NaN,NaN
11,112862-2018,112862,7,Manteuffelstraße ...,7400823,2018,1,16,4,3,...,7400823.0,Wittekindstraße,74008.0,Tempelhof Südwest,740.0,Tempelhof,POINT (13.37773 52.47010),NaN,NaN,NaN
18,112980-2018,112980,7,Manteuffelstraße ...,7400823,2018,1,13,3,3,...,7400823.0,Wittekindstraße,74008.0,Tempelhof Südwest,740.0,Tempelhof,POINT (13.37744 52.47012),NaN,NaN,NaN
21,113020-2018,113020,12,Ziekowstraße/Freie Scholle,12500824,2018,1,7,6,3,...,12500824.0,Ziekowstraße/Freie Scholle,125008.0,West 3 - Borsigwalde/Freie Scholle,1250.0,Wittenau/Borsigwalde,POINT (13.28410 52.59705),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50094,112216-2021,112216,4,0,4501153,2021,6,8,5,3,...,4501153.0,Babelsberger Straße,45011.0,Volkspark Wilmersdorf,450.0,Wilmersdorf Zentrum,POINT (13.33649 52.47839),NaN,NaN,NaN
50107,112189-2021,112189,4,0,4400830,2021,9,14,3,3,...,4400830.0,Forckenbeckstraße,44008.0,Schmargendorf,440.0,Wilmersdorf Süd,POINT (13.30653 52.48018),NaN,NaN,NaN
50108,112187-2021,112187,7,0,7400928,2021,3,15,1,3,...,7400928.0,Germaniagarten,74009.0,Tempelhof Südost,740.0,Tempelhof,POINT (13.41959 52.46106),NaN,NaN,NaN
50115,111541-2021,111541,4,0,4400726,2021,1,1,6,3,...,4400726.0,Bismarckallee,44007.0,Grunewald,440.0,Wilmersdorf Süd,POINT (13.28950 52.49381),NaN,NaN,NaN


In [72]:
# Set CRS for accident data to WGS 84
nan_entries_gdf = nan_entries_gdf.set_crs('EPSG:4326')

# Convert accident data to the same CRS as the street data
nan_entries_gdf = nan_entries_gdf.to_crs(STREETS_gdf.crs)

In [73]:
from shapely.ops import nearest_points
from tqdm import tqdm

# Initialize columns to store the nearest street information
nan_entries_gdf['nearest_street_idx'] = None
nan_entries_gdf['second_street_search'] = None

# For each accident point, find the nearest street
for index, row in tqdm(nan_entries_gdf.iterrows()):
    point = row.geometry
    
    # Find the nearest street geometry and its index
    nearest_geom, nearest_idx = None, None
    min_distance = float('inf')
    for idx, street_row in STREETS_gdf.iterrows():
        street_geom = street_row.geometry
        distance = point.distance(street_geom)
        if distance < min_distance:
            min_distance = distance
            nearest_geom = street_geom
            nearest_idx = idx
    
    # Update the GeoDataFrame with the nearest street information
    nan_entries_gdf.at[index, 'nearest_street_idx'] = nearest_idx
    nan_entries_gdf.at[index, 'second_street_search'] = STREETS_gdf.at[nearest_idx, 'strassenna']

0it [00:00, ?it/s]C:\Users\Domen\anaconda3\lib\site-packages\shapely\measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
1it [00:03,  3.22s/it]C:\Users\Domen\anaconda3\lib\site-packages\shapely\measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
2it [00:06,  3.21s/it]C:\Users\Domen\anaconda3\lib\site-packages\shapely\measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
3it [00:09,  3.28s/it]C:\Users\Domen\anaconda3\lib\site-packages\shapely\measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
4it [00:12,  3.25s/it]C:\Users\Domen\anaconda3\lib\site-packages\shapely\measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
5it [00:16,  3.22s/it]C:\Users\Domen\anaconda3\lib\site-packages\shapely\measurement

In [74]:
nan_entries_gdf.to_pickle("../data/temp/temp_adress_data2.pkl")
